In [1]:
import numpy as np
import pandas as pd
from statistics import mean

In [2]:
import sys
from os.path import abspath,join
sys.path.append(abspath(join('..', 'module')))
from openTable import *
from AccesDB import *

In [51]:
# df = pd.read_csv('../data/univ_sbm_with_rerata.csv')
# df['facultydepartmentid'] = df['facultydepartmentid'].astype('int')
# df['universityId'] = df['universityId'].astype('int')
# data dari drive
df = pd.read_excel(r'../data/Data_PTN_SBMPTN.xlsx')

In [52]:
df.columns

Index(['Kode Univ', 'Nama Universitas sbm', 'Jenis', 'Kode Prodi', 'Prodi',
       'DT 2020', 'P 2019', 'Portofolio', 'rataan', 's_baku', 'max', 'min',
       'Nama Universitas', 'Nama Prodi'],
      dtype='object')

In [53]:
keketatan = []
namagabungan = []
for index, row in df.iterrows():
    if row['P 2019'] > row['DT 2020']:
        ketat = (row['DT 2020']/row['P 2019'])*100
        keketatan.append(ketat)
    else:
        keketatan.append(float('NaN'))
    un = str(row['Nama Universitas'])
    pr = str(row['Nama Prodi'])
    gabung = un.replace(' ','') + pr.replace(' ','')
    namagabungan.append(gabung)
    
df['persentase_keketatan'] = keketatan
df['namagabungan'] = namagabungan
df['rataan'] = df['rataan'].str.replace(',','.')
df['s_baku'] = df['s_baku'].str.replace(',','.')
df['max'] = df['max'].str.replace(',','.')
df['min'] = df['min'].str.replace(',','.')
df['rataan'] = df['rataan'].astype('float')
df['s_baku'] = df['s_baku'].astype('float')
df['max'] = df['max'].astype('float')
df['min'] = df['min'].astype('float')

In [69]:
col = ['departmentId', 'departmentgroupId', 'departmentname']
tables = 'AP_Department'
server, koneksi = Connection()
AP_Department, status = open_table(koneksi, col, tables)
AP_Department = rename_column(AP_Department,{0:'departmentId', 1:'departmentgroupId', 2:'departmentname'})

col = ['departmentgroupId', 'departmentgroupname']
tables = 'AP_DepartmentGroup'
AP_DepartmentGroup, status = open_table(koneksi, col, tables)
AP_DepartmentGroup = rename_column(AP_DepartmentGroup,{0:'departmentgroupId', 1:'departmentgroupname'})

col = ['facultydepartmentid', 'facultyid', 'departmentid', 'accreditation']
tables = 'AP_FacultyDepartment'
AP_FacultyDepartment,status = open_table(koneksi, col, tables)
AP_FacultyDepartment = rename_column(AP_FacultyDepartment,{0:'facultydepartmentid', 1:'facultyid', 2:'departmentId', 
                                                           3:'accreditation'})

col = ['facultyid', 'universityid']
tables = 'AP_Faculty'
AP_Faculty,status = open_table(koneksi, col, tables)
AP_Faculty = rename_column(AP_Faculty,{0:'facultyid', 1:'universityId'})

col = ['universityId','name']
tables = 'AP_University'
server, koneksi = Connection()
AP_University,status = open_table(koneksi, col, tables)
AP_University = rename_column(AP_University,{0:'universityId', 1:'name'})
koneksi.close()
server.stop()

In [70]:
department = pd.merge(AP_Department,AP_DepartmentGroup,on='departmentgroupId')
faculty = pd.merge(AP_FacultyDepartment,AP_Faculty,on='facultyid')
univ = pd.merge(faculty,AP_University,on='universityId')
departmentgroup = pd.merge(department,univ,on='departmentId')
departmentgroup = departmentgroup[['universityId','facultydepartmentid','accreditation','name','departmentname']]

In [37]:
namagabungan = []
for index, row in departmentgroup.iterrows():
    uni = row['name'].lower()
    jur = row['departmentname'].lower()
    gabung = uni.replace(' ','')+jur.replace(' ','')
    namagabungan.append(gabung)
departmentgroup['namagabungan'] = namagabungan

In [38]:
departmentgroup = departmentgroup.drop_duplicates(subset=['name', 'departmentname'],keep=False)

In [39]:
data = pd.merge(df,departmentgroup,on='namagabungan', how='left')

In [40]:
data = data.drop(['Portofolio','max','min','name','departmentname'], axis=1)

In [41]:
# isi universityId yang NaN
universityId_baru = []
for i, row in data.iterrows():
    if np.isnan(row['universityId']):
        univ = data.loc[data.index == i]['Nama Universitas'].values[0]
        try:
            univId = data.loc[(data['Nama Universitas']==univ)&(data['universityId'].isnull()==False)]['universityId'].head(1).values[0]
            universityId_baru.append(univId)
        except:
            if univ == 'institut teknologi bandung (itb)':
                universityId_baru.append(64210)
            else:
                universityId_baru.append(row['universityId'])
    else:
        universityId_baru.append(row['universityId'])
data.drop(['universityId'], axis=1)
data['universityId'] = universityId_baru

In [42]:
data.head()

,Kode Univ,Nama Universitas sbm,Jenis,Kode Prodi,Prodi,DT 2020,P 2019,rataan,s_baku,Nama Universitas,Nama Prodi,persentase_keketatan,namagabungan,universityId,facultydepartmentid,accreditation
0,111,universitas syiah kuala,SAINTEK,1111014,pendidikan dokter hewan,99,712,NaN,NaN,universitas syiah kuala (unsyiah),dokter hewan,13.904494,universitassyiahkuala(unsyiah)dokterhewan,76394.0,NaN,NaN
1,111,universitas syiah kuala,SAINTEK,1111022,teknik sipil,81,753,NaN,NaN,universitas syiah kuala (unsyiah),teknik sipil,10.756972,universitassyiahkuala(unsyiah)tekniksipil,76394.0,181945.0,A
2,111,universitas syiah kuala,SAINTEK,1111037,teknik mesin,63,292,NaN,NaN,universitas syiah kuala (unsyiah),teknik mesin,21.575342,universitassyiahkuala(unsyiah)teknikmesin,76394.0,181949.0,A
3,111,universitas syiah kuala,SAINTEK,1111045,teknik kimia,63,296,NaN,NaN,universitas syiah kuala (unsyiah),teknik kimia,21.283784,universitassyiahkuala(unsyiah)teknikkimia,76394.0,181961.0,A
4,111,universitas syiah kuala,SAINTEK,1111053,arsitektur,63,487,NaN,NaN,universitas syiah kuala (unsyiah),arsitektur,12.936345,universitassyiahkuala(unsyiah)arsitektur,76394.0,182053.0,B


In [43]:
rataan_baru = []
s_baku_baru = []
for index, row in data.iterrows():
    if np.isnan(row['rataan']):
        if row['P 2019']>row['DT 2020']:
            try:
                jurusan = row['Prodi']
    #             akreditasi = row['accreditation']
                keketatan = row['persentase_keketatan']
                tabel = data.loc[(data['Prodi']==jurusan)]
                not_null = tabel[pd.notnull(tabel['rataan'])]
                mean_rataan = mean(not_null['rataan'].tolist())
                mean_s_baku = mean(not_null['s_baku'].tolist())
                mean_keketatan = mean(not_null['persentase_keketatan'].tolist())
                if keketatan > 100:
                    rataan = float('NaN')
                    s_baku = float('NaN')
                else:
                    rataan = ((100-keketatan)/(100-mean_keketatan))*mean_rataan
                    s_baku = ((100-keketatan)/(100-mean_keketatan))*mean_s_baku
            except Exception as e:
                rataan = float('NaN')
                s_baku = float('NaN')
        else:
            rataan = row['rataan']
            s_baku = row['s_baku']
    else:
        rataan = row['rataan']
        s_baku = row['s_baku']
    
    rataan_baru.append(rataan)
    s_baku_baru.append(s_baku)
    
data['rataan_baru'] = rataan_baru
data['s_baku_baru'] = s_baku_baru

In [44]:
data.loc[data['rataan_baru'].isnull()==False]

,Kode Univ,Nama Universitas sbm,Jenis,Kode Prodi,Prodi,DT 2020,P 2019,rataan,s_baku,Nama Universitas,Nama Prodi,persentase_keketatan,namagabungan,universityId,facultydepartmentid,accreditation,rataan_baru,s_baku_baru
0,111,universitas syiah kuala,SAINTEK,1111014,pendidikan dokter hewan,99,712,NaN,NaN,universitas syiah kuala (unsyiah),dokter hewan,13.904494,universitassyiahkuala(unsyiah)dokterhewan,76394.0,NaN,NaN,617.711528,19.912628
1,111,universitas syiah kuala,SAINTEK,1111022,teknik sipil,81,753,NaN,NaN,universitas syiah kuala (unsyiah),teknik sipil,10.756972,universitassyiahkuala(unsyiah)tekniksipil,76394.0,181945.0,A,602.087781,15.197186
2,111,universitas syiah kuala,SAINTEK,1111037,teknik mesin,63,292,NaN,NaN,universitas syiah kuala (unsyiah),teknik mesin,21.575342,universitassyiahkuala(unsyiah)teknikmesin,76394.0,181949.0,A,546.711972,13.670448
3,111,universitas syiah kuala,SAINTEK,1111045,teknik kimia,63,296,NaN,NaN,universitas syiah kuala (unsyiah),teknik kimia,21.283784,universitassyiahkuala(unsyiah)teknikkimia,76394.0,181961.0,A,556.654383,14.248741
4,111,universitas syiah kuala,SAINTEK,1111053,arsitektur,63,487,NaN,NaN,universitas syiah kuala (unsyiah),arsitektur,12.936345,universitassyiahkuala(unsyiah)arsitektur,76394.0,182053.0,B,599.997772,15.804207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3196,913,isbi tanah papua,SOSHUM,9132014,desain komunikasi visual,15,1,602.63,NaN,isbi tanah papua,desain komunikasi visual,NaN,isbitanahpapuadesainkomunikasivisual,53025552.0,53127500.0,NA,602.630000,NaN
3212,921,universitas papua,SAINTEK,9211247,teknik informatika,12,41,NaN,NaN,universitas negeri papua (unipa),teknik informatika,29.268293,universitasnegeripapua(unipa)teknikinformatika,75269.0,86372.0,B,471.495879,12.951858
3213,921,universitas papua,SAINTEK,9211255,teknik pertambangan,24,29,NaN,NaN,universitas negeri papua (unipa),teknik pertambangan,82.758621,universitasnegeripapua(unipa)teknikpertambangan,75269.0,86376.0,C,116.820672,3.250413
3223,921,universitas papua,SAINTEK,9211352,teknik sipil,16,19,NaN,NaN,universitas negeri papua (unipa),NaN,84.210526,universitasnegeripapua(unipa)nan,75269.0,NaN,NaN,106.525399,2.688788


In [18]:
# data.to_csv('../data/univ_sbm_with_rataan.csv')

In [47]:
data.to_excel('../data/univ_sbm_with_rataan.xlsx')

In [67]:
import sys
from os.path import abspath,join
sys.path.append(abspath(join('..', 'module')))

from openTable import *
from AccesDB import *
from filepath import *
from re import sub

def remove_slug(text):
    result = sub(r'\([^)]*\)', '', text)
    result = sub(r'["",]','', result)
    result = result.rstrip()
    result = result.replace(u'\xa0', u' ')
    result = " ".join(result.split())
    
    return result

def strip_space(text):
    result = "".join(text.split())
    
    return result

server,koneksi = Connection()

#Get Data Kampus
kampus,status = open_table(koneksi,['universityId','typeId','name'],'AP_University')
kampus = pd.DataFrame(kampus)
kampus = rename_column(kampus,{0:'universityId',1:'typeId',2:'Universitas'})

#Get Data Fakultas
fakultas,status = open_table(koneksi,['facultyid','universityid'],'AP_Faculty')
fakultas = pd.DataFrame(fakultas)
fakultas = rename_column(fakultas,{0:'facultyId',1:'universityId'})

#Get Data Department
department,status = open_table(koneksi,['departmentId','departmentname'],'AP_Department')
department = pd.DataFrame(department)
department = rename_column(department,{0:'departmentId',1:'Jurusan'})

#Get table aggregrate
agg,status = open_table(koneksi,['facultyid','departmentid','accreditation'],'AP_FacultyDepartment')
agg = pd.DataFrame(agg)
agg = rename_column(agg,{0:'facultyId',1:'departmentId',2:'accreditation'})
d = {'NA':'D'}
agg = agg.replace(d)

koneksi.close()
server.stop()

#merge universitasXfacultas (data1)
data = merge_table(kampus,fakultas,'universityId')

#merge data1Xjurusan
data = merge_table(data,agg,'facultyId')
data = merge_table(data,department,'departmentId')

#sort
data = data.sort_values(['universityId','accreditation'])

data = data.loc[data.typeId==62502]
data.Universitas = data.Universitas.str.lower()
data.Jurusan = data.Jurusan.str.lower()

data.Universitas = data.Universitas.apply(remove_slug)

data = data.replace({'Universitas': r'institut seni indonesia '}, {'Universitas': 'isi '}, regex=True) #ISI
data = data.replace({'Universitas': r'institut seni budaya indonesia '}, {'Universitas': 'isbi '}, regex=True) #ISBI
data = data.replace({'Universitas': r'universitas islam negeri alaudin makassar'}, {'Universitas': 'universitas islam negeri alauddin'}, regex=True) #alauddin
data = data.replace({'Universitas': r'universitas islam negeri ar raniry'}, {'Universitas': 'universitas islam negeri ar-raniry'}, regex=True) #ar raniry
data = data.replace({'Universitas': r'universitas islam negeri maulana malik ibrahim'}, {'Universitas': 'universitas islam negeri malang'}, regex=True) #maulana malik ibrahim
data = data.replace({'Universitas': r'universitas islam negeri sunan ampel surabaya'}, {'Universitas': 'universitas islam negeri sunan ampel'}, regex=True) #negeri sunan ampel
data = data.replace({'Universitas': r'universitas islam negeri sunan gunung djati bandung'}, {'Universitas': 'universitas islam negeri sunan gunung djati'}, regex=True) #sunan gunung djati
data = data.replace({'Universitas': r'universitas jendral soedirman'}, {'Universitas': 'universitas jenderal soedirman'}, regex=True) #jenderal soedirman
data = data.replace({'Universitas': r'universitas negeri papua'}, {'Universitas': 'universitas papua'}, regex=True) #universitas papua
data = data.replace({'Universitas': r'universitas singaperbangsa'}, {'Universitas': 'universitas singaperbangsa karawang'}, regex=True) #singaperbangsa
data = data.replace({'Universitas': r'universitas tidar magelang'}, {'Universitas': 'universitas tidar'}, regex=True) #tidar
data = data.replace({'Universitas': r'universitas trunojoyo'}, {'Universitas': 'universitas trunojoyo madura'}, regex=True) #trunojoyo
data = data.replace({'Universitas': r'universitas pembangunan nasioanl veteran jakarta'}, {'Universitas': 'universitas pembangunan nasional veteran jakarta'}, regex=True) #upn jakarta
data = data.replace({'Universitas': r'universitas islam negeri sumatra utara'}, {'Universitas': 'universitas islam negeri sumatera utara'}, regex=True) #uin sumut
data = data.replace({'Universitas': r'universitas islam negeri syarif hidayatullah'}, {'Universitas': 'universitas islam negeri jakarta'}, regex=True) #uin jkt
data = data.replace({'Universitas': r'universitas 19 november kolaka'}, {'Universitas': 'universitas sembilan belas november kolaka'}, regex=True) #uin jkt


# Make nama gabungan
data['nama gabungan'] = data.Universitas.str.cat(data.Jurusan.astype(str), sep=' ')
data['nama gabungan'] = data['nama gabungan'].apply(strip_space)

# data.to_csv('../data/database_ptn.csv',header=data.columns.tolist(),index=None)

In [68]:
data

,universityId,typeId,Universitas,facultyId,departmentId,accreditation,Jurusan,nama gabungan
0,62504,62502,institut teknologi sepuluh nopember,62514,60193,A,teknik mesin,institutteknologisepuluhnopemberteknikmesin
236,62504,62502,institut teknologi sepuluh nopember,62514,60533,A,teknik industri,institutteknologisepuluhnopemberteknikindustri
412,62504,62502,institut teknologi sepuluh nopember,62514,60204,A,teknik kimia,institutteknologisepuluhnopemberteknikkimia
485,62504,62502,institut teknologi sepuluh nopember,62514,60542,A,teknik fisika,institutteknologisepuluhnopemberteknikfisika
494,62504,62502,institut teknologi sepuluh nopember,62514,67150,A,teknik material,institutteknologisepuluhnopemberteknikmaterial
...,...,...,...,...,...,...,...,...
1076,53128794,62502,universitas timor,53158443,65031,D,teknologi informasi,universitastimorteknologiinformasi
1589,53128794,62502,universitas timor,53158443,60239,D,kimia,universitastimorkimia
1687,53128794,62502,universitas timor,53158443,60241,D,biologi,universitastimorbiologi
1771,53128794,62502,universitas timor,53158443,60221,D,matematika,universitastimormatematika
